# Dernier checkpoint - formation data analyst
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/h4r1c0t/WildCodeSchool/master?filepath=Odyssey%2FCheckpoint%2F%234_Final%2FCP4_sujet1.ipynb)
___
## Sujet n°1 : SQL Database
##### import

In [45]:
import pandas as pd
import requests
import sqlite3

import plotly.graph_objects as go

### Open dataset

In [2]:
# Get dataset from github.com
link = 'https://github.com/murpi/wilddata/raw/master/quests/chinook.db'
r = requests.get(link)
open('chinook.db', 'wb').write(r.content)

# Create connector
conn = sqlite3.connect('chinook.db')
cursor = conn.cursor()

### DB schemas

![schemas](http://images.innoveduc.fr/data/SQL/SQLite_chinook.png)

### 1. Mapping of sales by country, and comparison with customer and employee addresses
#### Explore dataset
##### Invoices table:

In [28]:
df_invoices = pd.read_sql("SELECT * FROM invoices", conn)
df_invoices.head(3)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94


In [31]:
# Columns filling rate:
round((1 - df_invoices.isna().sum()/len(df_invoices)) *100, 2)

InvoiceId            100.00
CustomerId           100.00
InvoiceDate          100.00
BillingAddress       100.00
BillingCity          100.00
BillingState          50.97
BillingCountry       100.00
BillingPostalCode     93.20
Total                100.00
dtype: float64

##### Customers table:

In [23]:
df_customers = pd.read_sql("SELECT * FROM customers", conn)
df_customers.head(3)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


In [32]:
# Columns filling rate:
round((1 - df_customers.isna().sum()/len(df_customers)) *100, 2)

CustomerId      100.00
FirstName       100.00
LastName        100.00
Company          16.95
Address         100.00
City            100.00
State            50.85
Country         100.00
PostalCode       93.22
Phone            98.31
Fax              20.34
Email           100.00
SupportRepId    100.00
dtype: float64

##### Employees table:

In [24]:
df_employees = pd.read_sql("SELECT * FROM employees", conn)
df_employees.head(3)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com


In [33]:
# Columns filling rate:
round((1 - df_employees.isna().sum()/len(df_employees)) *100, 2)

EmployeeId    100.0
LastName      100.0
FirstName     100.0
Title         100.0
ReportsTo      87.5
BirthDate     100.0
HireDate      100.0
Address       100.0
City          100.0
State         100.0
Country       100.0
PostalCode    100.0
Phone         100.0
Fax           100.0
Email         100.0
dtype: float64

#### Map creation

In [46]:
# Create a graph object 
salesMap = go.Figure()

# 